# RESNET for cifar10

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [4]:
import visdom
#python -m visdom.server
vis = visdom.Visdom()
vis.close(env = "main")

Setting up a new session...


''

**define value tracker**

In [5]:
def value_tracker(value_plot, value, num):
    vis.line(X=num, Y=value, win =value_plot, update='append')

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

**transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))**

**How to Calculate mean and std in Normalize**

In [7]:
transform = transforms.Compose([
    transforms.ToTensor()
])
#데이터 불러오기
trainset = torchvision.datasets.CIFAR10(root = './data', train=True, download=True, transform=transform)
print(trainset.data.shape)
#Normalize 하기
train_data_mean = trainset.data.mean(axis=(0,1,2))
train_data_std = trainset.data.std(axis=(0,1,2))
print(train_data_mean)
print(train_data_std)

train_data_mean = train_data_mean / 255
train_data_std = train_data_std / 255
print(train_data_mean)
print(train_data_std)

Files already downloaded and verified
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [8]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 256, 
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                        shuffle=True, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
import torchvision.models.resnet as resnet

In [10]:
conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

In [11]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 16
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=1,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        #x.shape = [batch, 3x32x32]
        x = self.conv1(x)
        # x.shape = [1, 16, 32, 32]
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        #x.shape = [1, 128, 32, 32]
        x = self.layer2(x)
        #x.shape = [1, 256, 32, 32]
        x = self.layer3(x)
        #x.shape = [1, 512, 16, 16]
        x = self.layer4(x)
        #x.shape = [1, 1024, 8, 8]
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

In [12]:
resnet50 = ResNet(Bottleneck, [3,4,6,3], 10, True).to(device)
#1(conv1) + 9(layer1) + 12(layer2) + 18(layer3) + 9(layer4) + 1(fc) = ResNet50

In [13]:
resnet50

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [16]:
a = torch.Tensor(1, 3, 32, 32).to(device)
out = resnet50(a)
print(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [17]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(resnet50.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

make plot

In [18]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts = dict(title = 'loss_tracker', legend = ['loss'], showlegend = True))
acc_plt = vis.line(Y=torch.Tensor(1).zero_(), opts = dict(title='acc_tracker', legend = ['acc'], showlegend =  True))

define acc_check function

In [24]:
def acc_check(net, test_set):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    acc = (correct/total *100)
    print("Accuracy of the network on the 10000 test images : %d %%" %acc)
    return acc

**Training with (acc check + model save)**

In [25]:
print(len(trainloader))
epochs = 150

for epoch in range(epochs):
    running_loss = 0.0
    lr_sche.step()
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = resnet50(inputs)
        loss = criterion(hypothesis, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i %30 == 29:
            value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader)]))
            print('[%d, %5d] loss : %.3f' %(epoch+1, i+1, running_loss/30))
            running_loss = 0.0
        
    acc = acc_check(resnet50, testloader)
    value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))

print('Finished Training')

196
[1,    30] loss : 1.514
[1,    60] loss : 1.451
[1,    90] loss : 1.413
[1,   120] loss : 1.400
[1,   150] loss : 1.334
[1,   180] loss : 1.275
Accuracy of the network on the 10000 test images : 50 %
[2,    30] loss : 1.201
[2,    60] loss : 1.154
[2,    90] loss : 1.126
[2,   120] loss : 1.110
[2,   150] loss : 1.050
[2,   180] loss : 1.052
Accuracy of the network on the 10000 test images : 58 %
[3,    30] loss : 0.974
[3,    60] loss : 0.973
[3,    90] loss : 0.968
[3,   120] loss : 0.935
[3,   150] loss : 0.920
[3,   180] loss : 0.898
Accuracy of the network on the 10000 test images : 60 %
[4,    30] loss : 0.886
[4,    60] loss : 0.847
[4,    90] loss : 0.873
[4,   120] loss : 0.839
[4,   150] loss : 0.818
[4,   180] loss : 0.793
Accuracy of the network on the 10000 test images : 65 %
[5,    30] loss : 0.763
[5,    60] loss : 0.753
[5,    90] loss : 0.771
[5,   120] loss : 0.732
[5,   150] loss : 0.726
[5,   180] loss : 0.750
Accuracy of the network on the 10000 test images : 7

[41,    60] loss : 0.014
[41,    90] loss : 0.017
[41,   120] loss : 0.017
[41,   150] loss : 0.017
[41,   180] loss : 0.017
Accuracy of the network on the 10000 test images : 86 %
[42,    30] loss : 0.016
[42,    60] loss : 0.016
[42,    90] loss : 0.019
[42,   120] loss : 0.015
[42,   150] loss : 0.016
[42,   180] loss : 0.014
Accuracy of the network on the 10000 test images : 86 %
[43,    30] loss : 0.014
[43,    60] loss : 0.012
[43,    90] loss : 0.012
[43,   120] loss : 0.013
[43,   150] loss : 0.014
[43,   180] loss : 0.014
Accuracy of the network on the 10000 test images : 86 %
[44,    30] loss : 0.013
[44,    60] loss : 0.011
[44,    90] loss : 0.012
[44,   120] loss : 0.012
[44,   150] loss : 0.015
[44,   180] loss : 0.011
Accuracy of the network on the 10000 test images : 86 %
[45,    30] loss : 0.012
[45,    60] loss : 0.013
[45,    90] loss : 0.012
[45,   120] loss : 0.016
[45,   150] loss : 0.011
[45,   180] loss : 0.013
Accuracy of the network on the 10000 test images : 

[81,    30] loss : 0.003
[81,    60] loss : 0.003
[81,    90] loss : 0.004
[81,   120] loss : 0.003
[81,   150] loss : 0.003
[81,   180] loss : 0.003
Accuracy of the network on the 10000 test images : 87 %
[82,    30] loss : 0.004
[82,    60] loss : 0.004
[82,    90] loss : 0.003
[82,   120] loss : 0.003
[82,   150] loss : 0.003
[82,   180] loss : 0.004
Accuracy of the network on the 10000 test images : 86 %
[83,    30] loss : 0.004
[83,    60] loss : 0.003
[83,    90] loss : 0.003
[83,   120] loss : 0.003
[83,   150] loss : 0.003
[83,   180] loss : 0.003
Accuracy of the network on the 10000 test images : 87 %
[84,    30] loss : 0.003
[84,    60] loss : 0.003
[84,    90] loss : 0.003
[84,   120] loss : 0.003
[84,   150] loss : 0.003
[84,   180] loss : 0.004
Accuracy of the network on the 10000 test images : 87 %
[85,    30] loss : 0.004
[85,    60] loss : 0.003
[85,    90] loss : 0.004
[85,   120] loss : 0.003
[85,   150] loss : 0.003
[85,   180] loss : 0.003
Accuracy of the network on

[120,    90] loss : 0.003
[120,   120] loss : 0.003
[120,   150] loss : 0.004
[120,   180] loss : 0.004
Accuracy of the network on the 10000 test images : 86 %
[121,    30] loss : 0.003
[121,    60] loss : 0.003
[121,    90] loss : 0.003
[121,   120] loss : 0.003
[121,   150] loss : 0.003
[121,   180] loss : 0.004
Accuracy of the network on the 10000 test images : 86 %
[122,    30] loss : 0.003
[122,    60] loss : 0.004
[122,    90] loss : 0.003
[122,   120] loss : 0.003
[122,   150] loss : 0.003
[122,   180] loss : 0.003
Accuracy of the network on the 10000 test images : 86 %
[123,    30] loss : 0.003
[123,    60] loss : 0.004
[123,    90] loss : 0.003
[123,   120] loss : 0.003
[123,   150] loss : 0.004
[123,   180] loss : 0.003
Accuracy of the network on the 10000 test images : 86 %
[124,    30] loss : 0.003
[124,    60] loss : 0.003
[124,    90] loss : 0.003
[124,   120] loss : 0.003
[124,   150] loss : 0.003
[124,   180] loss : 0.003
Accuracy of the network on the 10000 test images

Additional Accuracy Check

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
print('Accuracy of the network on the 10000 test images : %d %%' %(correct/total *100))